# Import Required Libraries

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard


# Define Dataset Path and Gesture Classes

In [ ]:
# Define dataset path
DATASET_PATH = "/content/drive/MyDrive/Colab Notebooks/dataset_dynamic"
GESTURE_CLASSES = {"pointing": 4, "waving": 5}  # Class Labels


# Load Dataset

In [ ]:
# Prepare dataset
sequences, labels = [], []

# Load dataset
for gesture, label in GESTURE_CLASSES.items():
    gesture_path = os.path.join(DATASET_PATH, gesture)
    for file in os.listdir(gesture_path):
        file_path = os.path.join(gesture_path, file)
        sequence = np.load(file_path)                 # Shape:(30, 39) per file (N = no. of keypoints)
        sequences.append(sequence)
        labels.append(label)


# Data Preprocessing

In [ ]:
# Convert to NumPy arrays
X = np.array(sequences)  #Features (30, 39)
y = np.array(labels)  #Labels (4, 5)

# Convert labels to categorical (One-hot encoding)
y = to_categorical(y, num_classes=6)          # Since max label = 5, we set num_classes = 6

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=42)

print(f"Training samples: {X_train.shape}, Testing samples: {X_test.shape}")

Training samples: (54, 30, 39), Testing samples: (6, 30, 39)


In [ ]:
# TensorBoard Callback for logging
log_dir = os.path.join("Logs")
tb_callback = TensorBoard(log_dir=log_dir)


# Define CNN-LSTM Model

In [ ]:
# Define CNN-LSTM Model
model = Sequential()

# CNN for feature extraction (Keeps Time Dimension)
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=(30, X.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2))

# LSTM for sequence modeling (Keep Time Dimension)
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))  # Last LSTM outputs final prediction

# Fully Connected Layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))                                         # Prevent Overfitting
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='softmax'))                       #6 output classes (0-5)


In [ ]:
# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


In [ ]:
# Display model architecture
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)                    │ (None, 30, 64)              │           7,552 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_4 (MaxPooling1D)       │ (None, 15, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_5 (Conv1D)                    │ (None, 15, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_5 (MaxPooling1D)       │ (None, 7, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 7, 64)               │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 7, 128)              │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 6)                   │             198 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 236,326 (923.15 KB)

 Trainable params: 236,326 (923.15 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=500, batch_size=16, callbacks=[tb_callback], validation_data=(X_test, y_test))

Epoch 1/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - categorical_accuracy: 0.4153 - loss: 1.7807 - val_categorical_accuracy: 0.5000 - val_loss: 1.7273
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - categorical_accuracy: 0.5000 - loss: 1.6932 - val_categorical_accuracy: 0.5000 - val_loss: 1.4801
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - categorical_accuracy: 0.5083 - loss: 1.3795 - val_categorical_accuracy: 0.5000 - val_loss: 1.1306
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - categorical_accuracy: 0.4947 - loss: 1.1555 - val_categorical_accuracy: 0.5000 - val_loss: 0.9660
Epoch 5/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - categorical_accuracy: 0.6317 - loss: 0.8989 - val_categorical_accuracy: 0.5000 - val_loss: 1.0056
Epoch 6/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - categorical_accuracy: 0.5199 - loss: 0.8396 - val_categorical_accuracy: 0.5000 - val_loss: 0.7015
Epoch 7/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - categorical_accuracy: 0.4519 - loss: 0.9404 - val

In [ ]:
# Save the trained model in .keras format
model.save("/content/drive/MyDrive/Colab Notebooks/model/dynamic_gesture_final.keras")
print("Model saved as dynamic_gesture_final.keras")

Model saved as dynamic_gesture_final.keras


In [ ]:
# Make predictions on test data
y_pred = model.predict(X_test)

# Print predicted and actual class labels
for i in range(5):  # Show first 5 test cases
    print(f"Predicted: {np.argmax(y_pred[i])}, Actual: {np.argmax(y_test[i])}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 933ms/step
Predicted: 4, Actual: 4
Predicted: 4, Actual: 4
Predicted: 5, Actual: 5
Predicted: 5, Actual: 5
Predicted: 4, Actual: 4
